In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from imblearn.over_sampling import RandomOverSampler



In [21]:
import warnings
warnings.filterwarnings("ignore")


In [22]:
df_ufil_pred = pd.read_csv("../data/unfilled.csv")

target = pd.read_csv("../data/filtered_df.csv")['ICU']

In [23]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Display the head of the dataframe with all columns
df_ufil_pred.head()

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,1,1,57,0.0,0,160.0,55.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,110.0,60.0,65.0,14.0,37.8,4890.0,4210000.0,11.4,132000.0,99.0,29.0,0.94,338.0,31.0,33.0,1.08,NaN,136.0,3.4,8.40,1.15,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
1,0,0,62,0.0,0,159.0,70.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,140.0,90.0,124.0,14.0,38.5,5240.0,4980000.0,14.2,165000.0,104.0,40.0,1.00,260.0,31.0,39.0,0.82,NaN,140.0,3.9,9.40,1.19,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
2,1,0,72,0.0,0,175.0,83.0,1,0,0,0,0,0,1,0,0,0,0,0,0.0,0,0,1,1,0.0,7.0,0.2,2.0,80.0,60.0,87.0,NaN,39.0,6000.0,3010000.0,9.4,123000.0,107.0,38.0,0.91,474.0,18.0,18.0,1.75,0.36,139.0,4.0,8.13,1.10,369.0,7.41,30.4,2.6,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,1.0
3,1,0,53,0.0,0,177.0,96.0,1,1,0,0,0,0,0,0,0,0,1,0,0.0,0,1,2,1,0.0,572000.0,12.0,7.0,125.0,85.0,NaN,NaN,40.4,7040.0,4770000.0,14.4,19000.0,230.0,NaN,3.22,NaN,NaN,80.0,1.94,1.02,122.0,3.5,NaN,1.10,407.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN
4,0,1,58,0.0,0,160.0,64.3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1,1,1,0.0,602100.0,13.5,7.0,107.0,40.0,125.0,16.0,40.0,2810.0,4460.0,14.5,58000.0,115.0,48.0,1.30,NaN,NaN,37.0,1.48,NaN,135.0,3.9,NaN,1.10,455.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN


In [24]:
df_ufil_pred.shape[1]

65

In [25]:
empty_value_count = df_ufil_pred.isna().sum().sum()
nb_total_values = df_ufil_pred.shape[0]*df_ufil_pred.shape[1]

In [26]:
print("count is : " + str(target.count()) + " & sum is "+str(target.sum()))
print("empty_value_% = "+str(empty_value_count / nb_total_values))

count is : 72 & sum is 14
empty_value_% = 0.12756410256410255


In [27]:
# Number of folds
num_folds = 6

In [28]:
from sklearn.model_selection import KFold
import random

while True:
    # Initialize the KFold splitter with a random state
    random_state = random.randint(1, 10000)
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)

    trains_df = []
    tests_df = []
    targets_train_df = []
    targets_test_df = []

    # Split the data into 6 folds
    for train_index, test_index in kf.split(df_ufil_pred):
        # Create dataframes for training and testing
        train_df = df_ufil_pred.iloc[train_index]
        test_df = df_ufil_pred.iloc[test_index]

        # Append the training and testing dataframes to lists
        trains_df.append(train_df)
        tests_df.append(test_df)

        # Split the target into training and testing
        target_train_df = target.iloc[train_index]
        target_test_df = target.iloc[test_index]

        # Append the target dataframes to lists
        targets_train_df.append(target_train_df)
        targets_test_df.append(target_test_df)

    # Check if all target_train and target_test have at least two classes
    all_have_two_classes = all(len(df.unique()) >= 2 for df in targets_train_df + targets_test_df)

    if all_have_two_classes:
        break


In [29]:
tests_df[0]

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
4,0,1,58,0.0,0,160.0,64.3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1,1,1,0.0,602100.0,13.50,7.0,107.0,40.0,125.0,16.0,40.0,2810.0,4460.0,14.5,58000.0,115.0,48.0,1.30,NaN,NaN,37.0,1.48,NaN,135.0,3.9,NaN,1.10,455.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN
10,1,0,43,0.0,1,164.0,65.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1,0,1,1,0.0,115940.0,2.20,3.0,110.0,70.0,87.0,14.0,36.8,4380.0,5270000.0,15.9,7000.0,145.0,44.0,0.90,394.0,65.0,94.0,2.20,1.0,134.0,3.9,8.8,1.07,298.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,1,0.0,1,0,0.0,2.0
12,1,1,29,0.0,0,186.0,73.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,4927.0,1.00,3.0,140.0,80.0,70.0,14.0,39.4,3280.0,4970000.0,13.6,51000.0,73.0,35.0,0.90,483.0,26.0,43.0,0.80,NaN,137.0,3.3,9.2,1.26,332.0,7.44,24.5,NaN,0,0,1.0,1.0,0.0,0,0,0.0,1,0,0.0,NaN
22,0,1,44,0.0,0,NaN,NaN,1,0,0,0,0,1,1,0,0,0,0,0,0.0,1,1,4,1,0.0,NaN,15.00,7.0,120.0,60.0,74.0,NaN,39.2,4350.0,3410.0,10.2,50000.0,132.0,92.0,2.05,1044.0,NaN,86.0,7.08,4.7,134.0,4.0,8.1,1.33,374.0,NaN,20.2,2.3,0,0,1.0,1.0,0.0,0,1,0.0,0,1,0.0,NaN
34,1,1,45,0.0,0,172.0,70.0,1,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,2200.0,0.06,7.0,110.0,80.0,120.0,22.0,39.5,2960.0,3890000.0,12.7,43000.0,90.0,39.0,0.90,709.0,75.0,43.0,1.60,0.7,127.0,3.8,7.2,1.40,232.0,7.46,25.3,2.0,1,0,1.0,1.0,1.0,0,1,0.0,1,0,0.0,NaN
36,1,1,67,0.0,0,178.0,90.0,1,1,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,1.0,NaN,NaN,5.0,155.0,95.0,130.0,14.0,36.0,5620.0,4410000.0,11.4,59000.0,173.0,42.0,1.40,338.0,56.0,45.0,NaN,NaN,137.0,2.9,8.3,1.30,883.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,1,1,1.0,NaN
44,0,1,31,0.0,0,160.0,45.0,1,0,1,1,0,1,0,0,0,1,0,0,0.0,0,0,2,1,0.0,120000.0,3.00,6.0,84.0,50.0,100.0,31.0,41.0,17040.0,3100000.0,8.8,38000.0,82.0,112.0,3.10,NaN,150.0,44.0,1.70,0.8,135.0,3.4,7.1,1.36,479.0,7.35,14.8,2.5,1,0,0.0,0.0,0.0,0,0,1.0,1,0,NaN,NaN
57,1,1,38,NaN,0,185.0,86.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1,0,1,1,0.0,17765.0,0.30,NaN,110.0,70.0,130.0,18.0,39.5,3460.0,5330000.0,16.1,23000.0,88.0,41.0,1.50,1154.0,41.0,31.0,4.00,2.2,134.0,3.3,NaN,1.04,862.0,7.43,23.2,2.3,1,0,0.0,0.0,0.0,0,0,0.0,0,0,NaN,NaN
60,0,0,55,0.0,0,NaN,NaN,0,0,0,0,0,1,1,0,0,0,1,0,1.0,1,0,5,1,0.0,167383.0,4.60,NaN,100.0,60.0,100.0,16.0,37.5,1950.0,3630000.0,11.8,31000.0,99.0,48.0,1.45,472.0,168.0,202.0,4.02,1.9,126.0,4.2,7.9,1.53,104.0,NaN,NaN,NaN,1,0,0.0,0.0,0.0,0,0,0.0,0,0,NaN,NaN
61,1,1,29,0.0,0,175.0,84.5,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,1462.0,0.01,NaN,120.0,70.0,94.0,NaN,36.6,5880.0,4440000.0,13.4,128000.0,64.0,17.0,0.90,364.0,28.0,40.0,2.90,1.1,138.0,3.4,9.1,1.27,578.0,7.43,27.4,0.6,1,0,1.0,1.0,1.0,0,0,0.0,1,0,NaN,NaN


**Filling method**

In [30]:
feature_info = pd.read_csv("../data/feature_info.csv")

In [31]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Define the data types for imputation
data_types = {
    'categorical': np.int64,  # Categorical data type
    'int': np.int64,          # Integer data type
    'float': np.float64       # Float data type
}

def data_fillingKNN(dataframe) : 


    ### Before everything, get rid of nan only columns 
    # Check if any column contains only empty values (NaNs)
    empty_columns = dataframe.columns[dataframe.isnull().all()]

    # Replace empty columns with zeros
    for column in empty_columns:
        dataframe[column] = 0
    ###

    # Identify the missing values in filtered_df
    missing_values = dataframe.isnull().sum()

    # Separate features based on their data types
    categorical_features = feature_info[feature_info['type'] == 'categorical']['feature'].tolist()
    int_features = feature_info[feature_info['type'] == 'int']['feature'].tolist()
    float_features = feature_info[feature_info['type'] == 'float']['feature'].tolist()

    # Impute missing values for each data type
    for data_type, features in [('categorical', categorical_features), ('int', int_features), ('float', float_features)]:
        # Filter features based on data type
        features_to_impute = [feature for feature in features if feature in missing_values.index and missing_values[feature] > 0]

        if len(features_to_impute) > 0:
            # Prepare the imputation array
            impute_array = dataframe[features_to_impute].values

            if data_type in ['categorical', 'int']:
                # Perform imputation for 'categorical' and 'int' features using median strategy
                imputer = KNNImputer(n_neighbors=3, weights='distance')
                imputed_values = imputer.fit_transform(impute_array)

                dataframe.loc[:, features_to_impute] = np.round(imputed_values).astype(data_types[data_type])
                
            elif data_type == 'float':
                # Perform imputation for 'float' features using mean strategy
                imputer = KNNImputer(n_neighbors=5, weights='distance')
                imputed_values = imputer.fit_transform(impute_array)

                dataframe.loc[:, features_to_impute] = imputed_values

    # Verify if any missing values remain in the DataFrame
    missing_values_after_imputation = dataframe.isnull().sum()
    missing_values_to_print = missing_values_after_imputation[missing_values_after_imputation != 0]
    if not missing_values_to_print.empty:
        print(f"There are still missing values in the DataFrame after imputation:\n{missing_values_to_print}")

    return dataframe

In [32]:
# Apply data_fillingKNN to each dataframe in trains_df and tests_df
for i in range(len(trains_df)):
    trains_df[i] = data_fillingKNN(trains_df[i])


for i in range(len(tests_df)):
    tests_df[i] = data_fillingKNN(tests_df[i])

In [33]:
def standardize(df) : 
    means = df.mean(axis=0)
    std = df.std(axis=0)

    std_df = (df - means) / std.where(std != 0, 1)

    return std_df

# Apply standardize to each dataframe in trains_df and tests_df
for i in range(len(trains_df)):
    trains_df[i] = standardize(trains_df[i])

for i in range(len(tests_df)):
    tests_df[i] = standardize(tests_df[i])

**Oversampling method**

In [34]:
def oversample(df, associated_target): 
    # Check if there's only one class in the target variable
    unique_classes = set(associated_target)
    if len(unique_classes) == 1:
        # Return the original data without oversampling
        return df, associated_target

    # Oversample the minority class in the training data
    oversampler = RandomOverSampler(sampling_strategy='not majority', random_state=42)
    df_resampled, associated_target_resampled = oversampler.fit_resample(df, associated_target)
    return df_resampled, associated_target_resampled


# Apply oversampling to each dataframe and target
for i in range(len(trains_df)):
    trains_df[i], targets_train_df[i] = oversample(trains_df[i], targets_train_df[i])

for i in range(len(tests_df)):
    tests_df[i], targets_test_df[i] = oversample(tests_df[i], targets_test_df[i])



In [35]:
trains_df[1]

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,0.727659,0.867157,0.338254,-0.184141,-0.184141,-1.832969,-1.747793,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,0.623506,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.498773,-0.633920,0.831531,-0.173181,-0.663904,-1.648863,-0.977912,-0.285736,-0.317619,0.270149,-0.378380,0.915135,-0.427933,-0.474926,-0.500804,-0.921547,-0.616123,-0.546130,-0.477282,-0.449342,0.086136,-0.841960,-0.299338,-0.186147,0.275680,0.270328,0.755597,-0.193761,-1.797485,-0.184141,-1.060099,-1.133974,-0.867157,3.288870,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,-0.467370
1,-1.351366,-1.133974,0.668796,-0.184141,-0.184141,-1.925233,-0.653191,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,0.623506,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.441790,-0.633920,0.831531,1.332738,1.569791,1.362992,-0.977912,0.229790,-0.209896,0.799288,0.898296,1.486731,-0.337676,-0.292625,-0.453320,-1.216724,-0.616123,-0.383106,-0.531766,-0.331447,0.913039,0.182126,1.098830,0.038070,0.191089,0.269338,1.109671,-0.034612,-1.797485,-0.184141,-1.060099,-1.133974,-0.867157,3.288870,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,0.357400
2,0.727659,-1.133974,1.329879,-0.184141,-0.184141,-0.449016,0.295464,0.991632,-0.521523,-0.227496,-0.227496,-0.298988,-1.577103,2.360555,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.667771,-0.787642,-0.798922,-1.679100,-0.663904,-0.525798,1.246722,0.598022,0.024017,-0.554482,-1.290291,0.759245,-0.283521,-0.325771,-0.524546,-0.406878,-0.923988,-0.953690,-0.336882,-0.523693,0.706313,0.386944,-0.676843,-0.466418,-0.680848,-0.676036,1.617108,0.262289,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,-1.292140
3,0.727659,-1.133974,0.073821,-0.184141,-0.184141,-0.264489,1.244119,0.991632,1.885505,-0.227496,-0.227496,-0.298988,-1.577103,-0.416569,-0.129099,-0.265025,-0.388950,2.528175,-0.184141,-0.265025,-0.495816,2.092917,0.182228,0.129099,-0.129099,1.967170,1.479763,0.831531,0.579779,1.197508,0.290976,1.246722,1.629074,0.344108,0.654978,0.989487,-1.042147,1.936802,-0.068091,1.303604,-1.065351,-0.687169,0.730890,-0.297067,-0.231911,-2.808026,-0.637143,-0.114307,-0.466418,-0.433583,-0.099412,-0.349751,0.396100,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,1.182171
4,-1.351366,0.867157,0.404363,-0.184141,-0.184141,-1.832969,-1.069140,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,-1.577103,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,2.092917,-0.598749,0.129099,-0.129099,2.105828,1.767992,0.831531,-0.323773,-2.153033,1.414040,-0.421753,1.334488,-0.957801,-2.619869,1.035082,-0.366625,-0.139110,-0.160042,-0.215898,1.311206,-0.450350,-0.437447,-0.393461,-0.256771,-0.120590,0.182126,0.291607,-0.466418,-0.121247,0.161091,0.666443,0.277010,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0

**Data augmentation**

In [36]:
def doubleWithNoise(df, target) :

    # Determine the number of samples to add (doubling the size)
    num_samples_to_add = len(df)
    num_columns = len(df.columns)

    # Generate random noise with mean 0 and variance 1
    noise = np.random.normal(0, 1, (num_samples_to_add, num_columns))

    # Randomly select samples from the original DataFrame
    random_indices = np.random.choice(len(df), num_samples_to_add, replace=True)
    selected_samples = df.iloc[random_indices]

    # Add noise to the selected samples
    new_samples = selected_samples + noise
    

    # Concatenate the original DataFrame with the new samples
    doubled_df = pd.concat([df, new_samples])
    doubled_target = np.concatenate([target, target[random_indices]])

    return doubled_df, doubled_target


for i in range(num_folds):
    trains_df[i], targets_train_df[i] = doubleWithNoise(trains_df[i], targets_train_df[i])


In [37]:
trains_df[1]

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,0.727659,0.867157,0.338254,-0.184141,-0.184141,-1.832969,-1.747793,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,0.623506,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.498773,-0.633920,0.831531,-0.173181,-0.663904,-1.648863,-0.977912,-0.285736,-0.317619,0.270149,-0.378380,0.915135,-0.427933,-0.474926,-0.500804,-0.921547,-0.616123,-0.546130,-0.477282,-0.449342,0.086136,-0.841960,-0.299338,-0.186147,0.275680,0.270328,0.755597,-0.193761,-1.797485,-0.184141,-1.060099,-1.133974,-0.867157,3.288870,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,-0.467370
1,-1.351366,-1.133974,0.668796,-0.184141,-0.184141,-1.925233,-0.653191,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,0.623506,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.441790,-0.633920,0.831531,1.332738,1.569791,1.362992,-0.977912,0.229790,-0.209896,0.799288,0.898296,1.486731,-0.337676,-0.292625,-0.453320,-1.216724,-0.616123,-0.383106,-0.531766,-0.331447,0.913039,0.182126,1.098830,0.038070,0.191089,0.269338,1.109671,-0.034612,-1.797485,-0.184141,-1.060099,-1.133974,-0.867157,3.288870,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,0.357400
2,0.727659,-1.133974,1.329879,-0.184141,-0.184141,-0.449016,0.295464,0.991632,-0.521523,-0.227496,-0.227496,-0.298988,-1.577103,2.360555,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,-0.469839,-0.598749,0.129099,-0.129099,-0.667771,-0.787642,-0.798922,-1.679100,-0.663904,-0.525798,1.246722,0.598022,0.024017,-0.554482,-1.290291,0.759245,-0.283521,-0.325771,-0.524546,-0.406878,-0.923988,-0.953690,-0.336882,-0.523693,0.706313,0.386944,-0.676843,-0.466418,-0.680848,-0.676036,1.617108,0.262289,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,-1.292140
3,0.727659,-1.133974,0.073821,-0.184141,-0.184141,-0.264489,1.244119,0.991632,1.885505,-0.227496,-0.227496,-0.298988,-1.577103,-0.416569,-0.129099,-0.265025,-0.388950,2.528175,-0.184141,-0.265025,-0.495816,2.092917,0.182228,0.129099,-0.129099,1.967170,1.479763,0.831531,0.579779,1.197508,0.290976,1.246722,1.629074,0.344108,0.654978,0.989487,-1.042147,1.936802,-0.068091,1.303604,-1.065351,-0.687169,0.730890,-0.297067,-0.231911,-2.808026,-0.637143,-0.114307,-0.466418,-0.433583,-0.099412,-0.349751,0.396100,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0.547061,-0.298988,-0.675049,-0.298988,-0.129099,1.182171
4,-1.351366,0.867157,0.404363,-0.184141,-0.184141,-1.832969,-1.069140,-0.991632,-0.521523,-0.227496,-0.227496,-0.298988,-1.577103,-0.416569,-0.129099,-0.265025,-0.388950,-0.388950,-0.184141,-0.265025,-0.495816,2.092917,-0.598749,0.129099,-0.129099,2.105828,1.767992,0.831531,-0.323773,-2.153033,1.414040,-0.421753,1.334488,-0.957801,-2.619869,1.035082,-0.366625,-0.139110,-0.160042,-0.215898,1.311206,-0.450350,-0.437447,-0.393461,-0.256771,-0.120590,0.182126,0.291607,-0.466418,-0.121247,0.161091,0.666443,0.277010,0.547061,-0.184141,0.927586,0.867157,1.133974,-0.298988,-0

**Evaluation**

In [38]:
from plot_risk_index import corr_risk_uncer

sum_corr = 0

for i in range(num_folds) : 
    corr = corr_risk_uncer(trains_df[i], targets_train_df[i], tests_df[i], targets_test_df[i])
    print(corr)
    sum_corr += corr

avg_corr = sum_corr / num_folds

print("avg corr = "+str(avg_corr))

Type of feature: <class 'pandas.core.series.Series'>
Type of train_target: <class 'numpy.ndarray'>
0     0.727659
1    -1.351366
2     0.727659
3     0.727659
4     0.727659
        ...   
0     0.836464
95   -0.390369
65    0.863215
25   -1.764573
6     1.403873
Name: Sesso, Length: 196, dtype: float64
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 0 0 1
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1
 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 1
 1 0 1 1 0 0 0 1 1 0 0]


TypeError: unsupported type: <class 'numpy.ndarray'>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

def evaluate(train_df_resampled, target_train_df_resampled, test_df_resampled, target_test_df_resampled) :
    # Initialize classifiers
    classifiers = {
        'Decision Tree (C4.5)': DecisionTreeClassifier(),
        'k-Nearest Neighbor (kNN)': KNeighborsClassifier(),
        'Logistic Regression (LR)': LogisticRegression(),
        'Naïve Bayes (NB)': GaussianNB(),
        'Random Forest (RF)': RandomForestClassifier(),
        'Support Vector Machine (SVM)': SVC(probability=True)
    }

    # Initialize lists to store AUC scores
    auc_scores = []

    # Iterate through the classifiers
    for classifier_name, classifier in classifiers.items():
        # print(f"Training and testing {classifier_name}...")
        
        # Initialize a list to store fold-wise AUC scores
        fold_auc_scores = []
        
        # Split data into train and test sets
        X_train = train_df_resampled
        y_train = target_train_df_resampled
        X_test = test_df_resampled
        y_test = target_test_df_resampled

        # Train the classifier on the training data
        classifier.fit(X_train, y_train)

        # Predict probabilities for the positive class
        y_prob = classifier.predict_proba(X_test)[:, 1]

        # Calculate AUC score for this fold
        auc = roc_auc_score(y_test, y_prob)

        # Append AUC score to fold-wise list
        fold_auc_scores.append(auc)

        # Calculate and print average AUC score across folds
        avg_auc = sum(fold_auc_scores) / num_folds
        
        # print(f"{classifier_name} - Average AUC Across {num_folds} Folds: {avg_auc:.2f}\n")
        
        # Append average AUC score to the overall list
        auc_scores.append(avg_auc)

    # Print overall AUC scores
    # print("Overall Average AUC Scores:")
    # for classifier_name, auc in zip(classifiers.keys(), auc_scores):
    #     print(f"{classifier_name}: {auc:.2f}")

    return np.mean(auc_scores)


In [ ]:
# Apply oversampling to each dataframe and target

sum_avg_auc = 0

for i in range(num_folds):
    auc = evaluate(trains_df[i], targets_train_df[i], tests_df[i], targets_test_df[i])
    print(auc)
    sum_avg_auc += auc

print("avg avg auc = "+str(sum_avg_auc / num_folds))

print("avg corr risk index = "+str(avg_corr))

0.13569444444444442
0.15277777777777776
0.13715277777777776
0.11522633744855966
0.1611111111111111
0.1586111111111111
avg avg auc = 0.14342892661179696


NameError: name 'avg_corr' is not defined